In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
!pip install torch transformers accelerate sentencepiece  


In [ ]:
# datasets en hugging face

from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

In [ ]:
# acceder a datos:

raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

In [ ]:
raw_train_dataset.features

In [ ]:
# reto: mirar el elemwento 15 del training set
# mirar el elemento 87 del validation set.

print(raw_train_dataset[15])
raw_val_dataset = raw_datasets["validation"]
print(raw_val_dataset[87])

In [ ]:
# tokenización a nivel de oración

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

In [ ]:
print(tokenized_sentences_1)

In [ ]:
print(tokenized_sentences_2)

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

In [ ]:
# reto 
# extraer el elemento 15 del training set y tokenicelo como sentencias separadas y como una pareja
# Cual es la diferencia entre los 2 resultados

raw_train_dataset = raw_datasets["train"]
stn1 = raw_train_dataset[15]['sentence1']
stn2 = raw_train_dataset[15]['sentence2']
print(stn1)
print(stn2)
inputs = tokenizer(stn1, stn2,stn1+stn2)
print(inputs)



In [ ]:
# de ids a tokens de nuevo

tokenizer.convert_ids_to_tokens(inputs["input_ids"])


In [ ]:
# tokenizar todo un dataset completo
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [ ]:
tokenized_dataset

In [ ]:
# fix padding
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['idx','sentence1','sentence2'])
tokenized_dataset = tokenized_dataset.rename_column('label','labels')
tokenized_dataset = tokenized_dataset.with_format('torch')
tokenized_dataset

In [ ]:
from torch.utils.data import DataLoader

train_dataloaders = DataLoader(tokenized_dataset['train'],batch_size=16,shuffle=True)

for step,batch in enumerate(train_dataloaders):
    print(batch['input_ids'].shape)
    if (step > 5):
        break

In [ ]:
# dynamic padding
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['idx','sentence1','sentence2'])
tokenized_dataset = tokenized_dataset.rename_column('label','labels')
tokenized_dataset = tokenized_dataset.with_format('torch')
tokenized_dataset

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
data_collactor = DataCollatorWithPadding(tokenizer)

train_dataloaders = DataLoader(tokenized_dataset['train'],batch_size=16,shuffle=True,collate_fn=data_collactor)

for step,batch in enumerate(train_dataloaders):
    print(batch['input_ids'].shape)
    if (step > 5):
        break

In [ ]:
samples = tokenized_dataset["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}